## 使用python实现Linear Regression算法

### 一、一元线性回归
一个简单的线性回归模型可以写成如下的形式：

y = b0 + b1 * x

### 1.计算均值和方差
首先，我们要计算训练集中的输入值和输出值的平均值和方差

In [1]:
def mean(values):
    return (sum(values)/float(len(values)))

def variance(values, mean):
    return sum([(x-mean)**2 for x in values])

使用mean和variance函数计算测试数据的均值和方差

In [2]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
x = [row[0] for row in dataset]
y = [row[1] for row in dataset]
mean_x, mean_y = mean(x), mean(y)
var_x, var_y = variance(x, mean_x), variance(y, mean_y) 
print('x stats: mean=%.3f variance=%.3f' % (mean_x, var_x))
print('y stats: mean=%.3f variance=%.3f' % (mean_y, var_y))

x stats: mean=3.000 variance=10.000
y stats: mean=2.800 variance=8.800


### 2.计算协方差
两组数字的协方差描述了这些数字如何一起变化。

协方差是相关性的泛化。 相关描述了两组数字之间的关系，而协方差可以描述两组或多组数字之间的关系。

In [3]:
def covariance(x, mean_x, y, mean_y):
    covar = 0.0
    for i in range(len(x)):
        covar += (x[i] - mean_x) * (y[i] - mean_y)
    return covar

使用covariance函数计算测试数据的协方差

In [4]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
x = [row[0] for row in dataset]
y = [row[1] for row in dataset]
mean_x, mean_y = mean(x), mean(y)
covar = covariance(x, mean_x, y, mean_y)
print ('Covariance: %.3f' % (covar))

Covariance: 8.000


### 3.计算回归方程中的系数b0和b1

In [5]:
def coefficients(dataset):
    x = [row[0] for row in dataset]
    y = [row[1] for row in dataset]
    mean_x, mean_y = mean(x), mean(y)
    b1 = covariance(x, mean_x, y, mean_y) / variance(x, mean_x)
    b0 = mean_y - b1 * mean_x
    return [b0, b1]

结合前面的mean、variance、covariance，使用coefficients计算测试数据中的回归系数

In [6]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
b0, b1 = coefficients(dataset)
print (('Coefficients: b0=%.3f, b1=%.3f' % (b0, b1)))

Coefficients: b0=0.400, b1=0.800


### 4.做出预测

In [7]:
def simple_linear_regression(train, test):
    predictions = list()
    b0, b1 = coefficients(train)
    for row in test:
        y = b0 + b1 * row[0]
        predictions.append(y)
    return predictions

最后增加两个函数，计算均方根误差和评估模型

In [8]:
import math

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        predict_error = predicted[i] = actual[i]
        sum_error += (predict_error ** 2)
    mean_error = sum_error / float(len(actual))
    return math.sqrt(mean_error)

In [9]:
def evaluate_algorithm(dataset, algorithm):
    test_set = []
    for row in dataset:
        row_copy = list(row)
        row_copy[-1] = None
        test_set.append(row_copy)
    predict = algorithm(dataset, test_set)
    actual = [row[-1] for row in dataset]
    rmse = rmse_metric(actual, predict)
    return rmse

### 二、多元线性回归(采用随机梯度下降优化)

多元线性回归的定义如下：

y = b0 + b1 * x1 + b2 * x2 + ...

随机梯度下降的定义如下：

b = b - learning_rate  *  error  *  x

### 1.作出预测
首先是要建立一个预测函数

In [10]:
def predict_mul(row, coeffcients):
    # 一般第一项都是截距
    y = coeffcients[0]
    for i in range(len(row)-1):
        y += coeffcients[i+1] * row[i]
    return y    

用测试数据来测试predict_mul函数

In [12]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.4, 0.8]
for row in dataset:
    y = predict_mul(row, coef)
    print (("Actual=%.3f, Predict=%.3f" % (row[-1], y)))

Actual=1.000, Predict=1.200
Actual=3.000, Predict=2.000
Actual=3.000, Predict=3.600
Actual=2.000, Predict=2.800
Actual=5.000, Predict=4.400


### 2.计算系数
我们在训练集中使用随机梯度下降方法计算系数

误差：

error = prediction - expected

随机梯度下降更新：

b(t+1) = b(t) - learning_rate * error(t) * x1(t)

In [13]:
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            y = predict_mul(row, coef)
            error = y - row[-1]
            sum_error += error ** 2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i+1] = coef[i+1] - l_rate * error * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return coef

使用测试数据通过随机梯度下降计算系数

In [14]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
l_rate = 0.001
n_epoch = 50
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print ((coef))

>epoch=0, lrate=0.001, error=46.236
>epoch=1, lrate=0.001, error=41.305
>epoch=2, lrate=0.001, error=36.930
>epoch=3, lrate=0.001, error=33.047
>epoch=4, lrate=0.001, error=29.601
>epoch=5, lrate=0.001, error=26.543
>epoch=6, lrate=0.001, error=23.830
>epoch=7, lrate=0.001, error=21.422
>epoch=8, lrate=0.001, error=19.285
>epoch=9, lrate=0.001, error=17.389
>epoch=10, lrate=0.001, error=15.706
>epoch=11, lrate=0.001, error=14.213
>epoch=12, lrate=0.001, error=12.888
>epoch=13, lrate=0.001, error=11.712
>epoch=14, lrate=0.001, error=10.668
>epoch=15, lrate=0.001, error=9.742
>epoch=16, lrate=0.001, error=8.921
>epoch=17, lrate=0.001, error=8.191
>epoch=18, lrate=0.001, error=7.544
>epoch=19, lrate=0.001, error=6.970
>epoch=20, lrate=0.001, error=6.461
>epoch=21, lrate=0.001, error=6.009
>epoch=22, lrate=0.001, error=5.607
>epoch=23, lrate=0.001, error=5.251
>epoch=24, lrate=0.001, error=4.935
>epoch=25, lrate=0.001, error=4.655
>epoch=26, lrate=0.001, error=4.406
>epoch=27, lrate=0.001,

### 一元线性回归完整代码如下所示：

In [ ]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def train_test_split(dataset, split):
    train = []
    train_size = split * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
    return train, dataset_copy
        
# 计算平方根平均误差
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

# 使用训练集评估线性回归算法
def evaluate_algorithm(dataset, algorithm):
    test_set = []
    for row in dataset:
        row_copy = list(row)
        row_copy[-1] = None
        test_set.append(row_copy)
    predicted = algorithm(dataset, test_set)
    print (predicted)
    actual = [row[-1] for row in dataset]
    rmse = rmse_metric(actual, predicted)
    return rmse

def mean(values):
    return (sum(values)/float(len(values)))

def variance(values, mean):
    return sum([(x-mean)**2 for x in values])

def covariance(x, mean_x, y, mean_y):
    covar = 0.0
    for i in range(len(x)):
        covar += (x[i] - mean_x) * (y[i] - mean_y)
    return covar

def coefficients(dataset):
    x = [row[0] for row in dataset]
    y = [row[1] for row in dataset]
    mean_x, mean_y = mean(x), mean(y)
    b1 = covariance(x, mean_x, y, mean_y) / variance(x, mean_x)
    b0 = mean_y - b1 * mean_x
    return [b0, b1]

def simple_linear_regression(train, test):
    predictions = []
    b0, b1 = coefficients(train)
    for row in test:
        y = b0 + b1 * row[0]
        predictions.append(y)
    return predictions

seed(1)
filename = 'insurance.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_to_float(dataset, i)
split = 0.66
rmse = evaluate_algorithm(dataset, simple_linear_regression, split)
print('RMSE: %.3f' % (rmse))

### 多元线性回归完整代码如下所示

In [ ]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
def find_minmax(dataset):
    minmax = []
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# 将每列的数据放缩到0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

def predict_mul(row, coeffcients):
    # 一般第一项都是截距
    y = coeffcients[0]
    for i in range(len(row)-1):
        y += coeffcients[i+1] * row[i]
    return y 

def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            y = predict_mul(row, coef)
            error = y - row[-1]
            sum_error += error ** 2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i+1] = coef[i+1] - l_rate * error * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return coef

def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = []
    coef = coefficients_sgd(train, l_rate, n_epoch)
    for row in test:
        y = predict_mul(row, coef)
        predictions.append(y)
    return (predictions)

seed(1)
filename = 'winequality-white.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_to_float(dataset, i)
    
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# 评估算法
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))